<a href="https://colab.research.google.com/github/Allen123321/DEMO-DL/blob/master/heart_beat_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 特征工程
+ 学习时间序列数据的特征预处理方法
+ 学习时间序列特征处理工具 Tsfresh（TimeSeries Fresh）的使用

+ 数据预处理
  + 时间序列数据格式处理
  + 加入时间步特征time
+ 特征工程
  + 时间序列特征构造
  + 特征筛选
  + 使用 tsfresh 进行时间序列特征处理

## 代码

In [1]:
!pip install tsfresh

In [2]:
!pip install scipy==1.5

In [3]:
# 导入包并读取数据
import numpy as np
import pandas as pd
import tsfresh as tsf
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# 数据读取
data_train = pd.read_csv("/content/drive/MyDrive/dataset/heartbeat-data/train.csv")
data_test_A = pd.read_csv("/content/drive/MyDrive/dataset/heartbeat-data/testA.csv")

print(data_train.shape)
print(data_test_A.shape)

(100000, 3)
(20000, 2)


In [5]:
data_train.head()

,id,heartbeat_signals,label
0,0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0


In [6]:
data_test_A.head()

,id,heartbeat_signals
0,100000,"0.9915713654170097,1.0,0.6318163407681274,0.13..."
1,100001,"0.6075533139615096,0.5417083883163654,0.340694..."
2,100002,"0.9752726292239277,0.6710965234906665,0.686758..."
3,100003,"0.9956348033996116,0.9170249621481004,0.521096..."
4,100004,"1.0,0.8879490481178918,0.745564725322326,0.531..."


## 数据预处理

In [7]:
# 对心电特征进行行转列处理，同时为每个心电信号加入时间步特征time
train_heartbeat_df = data_train["heartbeat_signals"].str.split(",", expand=True).stack()


train_heartbeat_df

0      0      0.9912297987616655
       1      0.9435330436439665
       2      0.7646772997256593
       3      0.6185708990212999
       4      0.3796321642826237
                     ...        
99999  200                   0.0
       201                   0.0
       202                   0.0
       203                   0.0
       204                   0.0
Length: 20500000, dtype: object

In [8]:
train_heartbeat_df = train_heartbeat_df.reset_index()
train_heartbeat_df

,level_0,level_1,0
0,0,0,0.9912297987616655
1,0,1,0.9435330436439665
2,0,2,0.7646772997256593
3,0,3,0.6185708990212999
4,0,4,0.3796321642826237
...,...,...,...
20499995,99999,200,0.0
20499996,99999,201,0.0
20499997,99999,202,0.0
20499998,99999,203,0.0


In [9]:
train_heartbeat_df = train_heartbeat_df.set_index("level_0")
train_heartbeat_df

,level_1,0
level_0,,
0,0,0.9912297987616655
0,1,0.9435330436439665
0,2,0.7646772997256593
0,3,0.6185708990212999
0,4,0.3796321642826237
...,...,...
99999,200,0.0
99999,201,0.0
99999,202,0.0


In [10]:
train_heartbeat_df.index.name = None
train_heartbeat_df

,level_1,0
0,0,0.9912297987616655
0,1,0.9435330436439665
0,2,0.7646772997256593
0,3,0.6185708990212999
0,4,0.3796321642826237
...,...,...
99999,200,0.0
99999,201,0.0
99999,202,0.0
99999,203,0.0


In [11]:
train_heartbeat_df.rename(columns={"level_1":"time", 0:"heartbeat_signals"}, inplace=True)
train_heartbeat_df["heartbeat_signals"] = train_heartbeat_df["heartbeat_signals"].astype(float)

In [12]:
train_heartbeat_df

,time,heartbeat_signals
0,0,0.991230
0,1,0.943533
0,2,0.764677
0,3,0.618571
0,4,0.379632
...,...,...
99999,200,0.000000
99999,201,0.000000
99999,202,0.000000
99999,203,0.000000


In [13]:
# 将处理后的心电特征加入到训练数据中，同时将训练数据label列单独存储
data_train_label = data_train["label"]


In [14]:
data_train_label

0        0.0
1        0.0
2        2.0
3        0.0
4        2.0
        ... 
99995    0.0
99996    2.0
99997    3.0
99998    2.0
99999    0.0
Name: label, Length: 100000, dtype: float64

In [15]:
data_train = data_train.drop("label", axis=1)

,id,heartbeat_signals
0,0,"0.9912297987616655,0.9435330436439665,0.764677..."
1,1,"0.9714822034884503,0.9289687459588268,0.572932..."
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23..."
3,3,"0.9757952826275774,0.9340884687738161,0.659636..."
4,4,"0.0,0.055816398940721094,0.26129357194994196,0..."
...,...,...
99995,99995,"1.0,0.677705342021188,0.22239242747868546,0.25..."
99996,99996,"0.9268571578157265,0.9063471198026871,0.636993..."
99997,99997,"0.9258351628306013,0.5873839035878395,0.633226..."
99998,99998,"1.0,0.9947621698382489,0.8297017704865509,0.45..."


In [16]:
data_train = data_train.drop("heartbeat_signals", axis=1)


data_train

,id
0,0
1,1
2,2
3,3
4,4
...,...
99995,99995
99996,99996
99997,99997
99998,99998


In [17]:
data_train = data_train.join(train_heartbeat_df)

data_train

,id,time,heartbeat_signals
0,0,0,0.991230
0,0,1,0.943533
0,0,2,0.764677
0,0,3,0.618571
0,0,4,0.379632
...,...,...,...
99999,99999,200,0.000000
99999,99999,201,0.000000
99999,99999,202,0.000000
99999,99999,203,0.000000


In [26]:
data_train[data_train["id"]==1]

,id,time,heartbeat_signals
1,1,0,0.971482
1,1,1,0.928969
1,1,2,0.572933
1,1,3,0.178457
1,1,4,0.122962
...,...,...,...
1,1,200,0.000000
1,1,201,0.000000
1,1,202,0.000000
1,1,203,0.000000


In [27]:
data_train

,id,time,heartbeat_signals
0,0,0,0.991230
0,0,1,0.943533
0,0,2,0.764677
0,0,3,0.618571
0,0,4,0.379632
...,...,...,...
99999,99999,200,0.000000
99999,99999,201,0.000000
99999,99999,202,0.000000
99999,99999,203,0.000000


In [32]:
from tsfresh import extract_features

# 特征提取
train_features = extract_features(data_train[:20499], column_id='id', column_sort='time')
train_features







Feature Extraction:   0%|          | 0/5 [00:00<?, ?it/s]





Feature Extraction:  20%|██        | 1/5 [00:06<00:27,  6.97s/it]





Feature Extraction:  40%|████      | 2/5 [00:12<00:20,  6.68s/it]





Feature Extraction:  60%|██████    | 3/5 [00:19<00:13,  6.68s/it]





Feature Extraction:  80%|████████  | 4/5 [00:26<00:06,  6.67s/it]





Feature Extraction: 100%|██████████| 5/5 [00:33<00:00,  6.62s/it]


,heartbeat_signals__variance_larger_than_standard_deviation,heartbeat_signals__has_duplicate_max,heartbeat_signals__has_duplicate_min,heartbeat_signals__has_duplicate,heartbeat_signals__sum_values,heartbeat_signals__abs_energy,heartbeat_signals__mean_abs_change,heartbeat_signals__mean_change,heartbeat_signals__mean_second_derivative_central,heartbeat_signals__median,heartbeat_signals__mean,heartbeat_signals__length,heartbeat_signals__standard_deviation,heartbeat_signals__variation_coefficient,heartbeat_signals__variance,heartbeat_signals__skewness,heartbeat_signals__kurtosis,heartbeat_signals__root_mean_square,heartbeat_signals__absolute_sum_of_changes,heartbeat_signals__longest_strike_below_mean,heartbeat_signals__longest_strike_above_mean,heartbeat_signals__count_above_mean,heartbeat_signals__count_below_mean,heartbeat_signals__last_location_of_maximum,heartbeat_signals__first_location_of_maximum,heartbeat_signals__last_location_of_minimum,heartbeat_signals__first_location_of_minimum,heartbeat_signals__percentage_of_reoccurring_values_to_all_values,heartbeat_signals__percentage_of_reoccurring_datapoints_to_all_datapoints,heartbeat_signals__sum_of_reoccurring_values,heartbeat_signals__sum_of_reoccurring_data_points,heartbeat_signals__ratio_value_number_to_time_series_length,heartbeat_signals__sample_entropy,heartbeat_signals__maximum,heartbeat_signals__minimum,heartbeat_signals__benford_correlation,heartbeat_signals__time_reversal_asymmetry_statistic__lag_1,heartbeat_signals__time_reversal_asymmetry_statistic__lag_2,heartbeat_signals__time_reversal_asymmetry_statistic__lag_3,heartbeat_signals__c3__lag_1,...,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_4,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_5,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_6,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_7,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_8,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_9,heartbeat_signals__ratio_beyond_r_sigma__r_0.5,heartbeat_signals__ratio_beyond_r_sigma__r_1,heartbeat_signals__ratio_beyond_r_sigma__r_1.5,heartbeat_signals__ratio_beyond_r_sigma__r_2,heartbeat_signals__ratio_beyond_r_sigma__r_2.5,heartbeat_signals__ratio_beyond_r_sigma__r_3,heartbeat_signals__ratio_beyond_r_sigma__r_5,heartbeat_signals__ratio_beyond_r_sigma__r_6,heartbeat_signals__ratio_beyond_r_sigma__r_7,heartbeat_signals__ratio_beyond_r_sigma__r_10,heartbeat_signals__count_above__t_0,heartbeat_signals__count_below__t_0,heartbeat_signals__lempel_ziv_complexity__bins_2,heartbeat_signals__lempel_ziv_complexity__bins_3,heartbeat_signals__lempel_ziv_complexity__bins_5,heartbeat_signals__lempel_ziv_complexity__bins_10,heartbeat_signals__lempel_ziv_complexity__bins_100,heartbeat_signals__fourier_entropy__bins_2,heartbeat_signals__fourier_entropy__bins_3,heartbeat_signals__fourier_entropy__bins_5,heartbeat_signals__fourier_entropy__bins_10,heartbeat_signals__fourier_entropy__bins_100,heartbeat_signals__permutation_entropy__dimension_3__tau_1,heartbeat_signals__permutation_entropy__dimension_4__tau_1,heartbeat_signals__permutation_entropy__dimension_5__tau_1,heartbeat_signals__permutation_entropy__dimension_6__tau_1,heartbeat_signals__permutation_entropy__dimension_7__tau_1,heartbeat_signals__query_similarity_count__query_None__threshold_0.0,"heartbeat_signals__matrix_profile__feature_""min""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""max""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""mean""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""median""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""25""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,1.0,1.0,38.927945,18.216197,0.019894,-0.004859,0.000117,0.125531,0.189892,205.0,0.229783,1.210071,0.052800,1.349485,1.90860

特征选择 train_features中包含了heartbeat_signals的779种常见的时间序列特征（所有这些特征的解释可以去看官方文档），这其中有的特征可能为NaN值（产生原因为当前数据不支持此类特征的计算），使用以下方式去除NaN值：

In [33]:
from tsfresh.utilities.dataframe_functions import impute

# 去除抽取特征中的NaN值
impute(train_features)

/usr/local/lib/python3.7/dist-packages/tsfresh/utilities/dataframe_functions.py:172: RuntimeWarning: The columns ['heartbeat_signals__friedrich_coefficients__coeff_0__m_3__r_30'
 'heartbeat_signals__friedrich_coefficients__coeff_1__m_3__r_30'
 'heartbeat_signals__friedrich_coefficients__coeff_2__m_3__r_30'
 'heartbeat_signals__friedrich_coefficients__coeff_3__m_3__r_30'
 'heartbeat_signals__max_langevin_fixed_point__m_3__r_30'
 'heartbeat_signals__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  df.iloc[:, np.where(is_col_non_finite)[0]].columns.values), RuntimeWarning)


,heartbeat_signals__variance_larger_than_standard_deviation,heartbeat_signals__has_duplicate_max,heartbeat_signals__has_duplicate_min,heartbeat_signals__has_duplicate,heartbeat_signals__sum_values,heartbeat_signals__abs_energy,heartbeat_signals__mean_abs_change,heartbeat_signals__mean_change,heartbeat_signals__mean_second_derivative_central,heartbeat_signals__median,heartbeat_signals__mean,heartbeat_signals__length,heartbeat_signals__standard_deviation,heartbeat_signals__variation_coefficient,heartbeat_signals__variance,heartbeat_signals__skewness,heartbeat_signals__kurtosis,heartbeat_signals__root_mean_square,heartbeat_signals__absolute_sum_of_changes,heartbeat_signals__longest_strike_below_mean,heartbeat_signals__longest_strike_above_mean,heartbeat_signals__count_above_mean,heartbeat_signals__count_below_mean,heartbeat_signals__last_location_of_maximum,heartbeat_signals__first_location_of_maximum,heartbeat_signals__last_location_of_minimum,heartbeat_signals__first_location_of_minimum,heartbeat_signals__percentage_of_reoccurring_values_to_all_values,heartbeat_signals__percentage_of_reoccurring_datapoints_to_all_datapoints,heartbeat_signals__sum_of_reoccurring_values,heartbeat_signals__sum_of_reoccurring_data_points,heartbeat_signals__ratio_value_number_to_time_series_length,heartbeat_signals__sample_entropy,heartbeat_signals__maximum,heartbeat_signals__minimum,heartbeat_signals__benford_correlation,heartbeat_signals__time_reversal_asymmetry_statistic__lag_1,heartbeat_signals__time_reversal_asymmetry_statistic__lag_2,heartbeat_signals__time_reversal_asymmetry_statistic__lag_3,heartbeat_signals__c3__lag_1,...,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_4,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_5,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_6,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_7,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_8,heartbeat_signals__energy_ratio_by_chunks__num_segments_10__segment_focus_9,heartbeat_signals__ratio_beyond_r_sigma__r_0.5,heartbeat_signals__ratio_beyond_r_sigma__r_1,heartbeat_signals__ratio_beyond_r_sigma__r_1.5,heartbeat_signals__ratio_beyond_r_sigma__r_2,heartbeat_signals__ratio_beyond_r_sigma__r_2.5,heartbeat_signals__ratio_beyond_r_sigma__r_3,heartbeat_signals__ratio_beyond_r_sigma__r_5,heartbeat_signals__ratio_beyond_r_sigma__r_6,heartbeat_signals__ratio_beyond_r_sigma__r_7,heartbeat_signals__ratio_beyond_r_sigma__r_10,heartbeat_signals__count_above__t_0,heartbeat_signals__count_below__t_0,heartbeat_signals__lempel_ziv_complexity__bins_2,heartbeat_signals__lempel_ziv_complexity__bins_3,heartbeat_signals__lempel_ziv_complexity__bins_5,heartbeat_signals__lempel_ziv_complexity__bins_10,heartbeat_signals__lempel_ziv_complexity__bins_100,heartbeat_signals__fourier_entropy__bins_2,heartbeat_signals__fourier_entropy__bins_3,heartbeat_signals__fourier_entropy__bins_5,heartbeat_signals__fourier_entropy__bins_10,heartbeat_signals__fourier_entropy__bins_100,heartbeat_signals__permutation_entropy__dimension_3__tau_1,heartbeat_signals__permutation_entropy__dimension_4__tau_1,heartbeat_signals__permutation_entropy__dimension_5__tau_1,heartbeat_signals__permutation_entropy__dimension_6__tau_1,heartbeat_signals__permutation_entropy__dimension_7__tau_1,heartbeat_signals__query_similarity_count__query_None__threshold_0.0,"heartbeat_signals__matrix_profile__feature_""min""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""max""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""mean""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""median""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""25""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""75""__threshold_0.98"
0,0.0,0.0,1.0,1.0,38.927945,18.216197,0.019894,-0.004859,0.000117,0.125531,0.189892,205.0,0.229783,1.210071,0.052800,1.349485,1.90860

In [39]:
from tsfresh import select_features

# 按照特征和数据label之间的相关性进行特征选择
train_features_filtered = select_features(train_features, data_train_label[:100])

train_features_filtered

,heartbeat_signals__percentage_of_reoccurring_values_to_all_values,heartbeat_signals__percentage_of_reoccurring_datapoints_to_all_datapoints,"heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""var""",heartbeat_signals__ratio_value_number_to_time_series_length,"heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""var""",heartbeat_signals__autocorrelation__lag_3,heartbeat_signals__autocorrelation__lag_2,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_49","heartbeat_signals__cwt_coefficients__coeff_1__w_2__widths_(2, 5, 10, 20)",heartbeat_signals__autocorrelation__lag_4,heartbeat_signals__partial_autocorrelation__lag_1,heartbeat_signals__autocorrelation__lag_1,heartbeat_signals__fourier_entropy__bins_100,"heartbeat_signals__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","heartbeat_signals__fft_aggregated__aggtype_""centroid""",heartbeat_signals__cid_ce__normalize_True,heartbeat_signals__number_crossing_m__m_0,heartbeat_signals__ratio_beyond_r_sigma__r_1,"heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""mean""","heartbeat_signals__fft_aggregated__aggtype_""skew""","heartbeat_signals__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""",heartbeat_signals__autocorrelation__lag_5,"heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""min""","heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_10__f_agg_""min""",heartbeat_signals__sum_of_reoccurring_values,heartbeat_signals__sum_of_reoccurring_data_points,"heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_50__f_agg_""var""","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_25","heartbeat_signals__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_55",heartbeat_signals__autocorrelation__lag_6,"heartbeat_signals__agg_linear_trend__attr_""stderr""__chunk_len_5__f_agg_""mean""",heartbeat_signals__binned_entropy__max_bins_10,heartbeat_signals__kurtosis,heartbeat_signals__maximum,"heartbeat_signals__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0",heartbeat_signals__range_count__max_1__min_-1,heartbeat_signals__value_count__value_1,"heartbeat_signals__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.6",heartbeat_signals__cid_ce__normalize_False,"heartbeat_signals__fft_coefficient__attr_""real""__coeff_0",heartbeat_signals__mean,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_0","heartbeat_signals__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0",heartbeat_signals__sum_values,"heartbeat_signals__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""var""",heartbeat_signals__approximate_entropy__m_2__r_0.5,"heartbeat_signals__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.2",heartbeat_signals__c3__lag_3,"heartbeat_signals__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""",heartbeat_signals__c3__lag_2,"heartbeat_signals__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0",heartbeat_signals__autocorrelation__lag_7,heartbeat_signals__mean_change,heartbeat_signals__ratio_beyond_r_sigma__r_3,heartbeat_signals__c3__lag_1,"heartbeat_signals__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.4",heartbeat_signals__abs_energy,heartbeat_signals__root_mean_square,"heartbeat_signals__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.4","heartbeat_signals__cwt_coefficients__coeff_5__w_2__widths_(2, 5, 10, 20)","heartbeat_signals__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""var""","heartbeat_signals__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__change_quantiles__f_agg_""mean""__isabs_False__qh_0.8__ql_0.0","heartbeat_signals__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.6",heartbeat_signals__lempel_ziv_complexity__bins_10,"heartbeat_signals__fft_coefficient__attr_"